# CMB-S4: DRAFT tool

## loading necessary python modules

In [1]:
%load_ext autoreload
%autoreload 2

#%pylab notebook
%matplotlib inline
from pylab import *
import os

import argparse, sys, numpy as np, scipy as sc, warnings, os
sys.path.append('modules/')
import tools

#import matplotlib.cbook
warnings.filterwarnings('ignore',category=RuntimeWarning)
#warnings.filterwarnings('ignore', category=DeprecationWarning)
#warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)

In [2]:
fsky = 0.57
fix_params = ['mnu']
prior_dict = {'tau': 0.007}
results_folder = 'results/'
ilc_pgm_name = 'get_ilc_weights_and_residuals.py'
fisher_pgm_name = 'get_fisher_withdelensing.py'
ilc_opfname = '%s/s4_cmb_ilc.npy' %(results_folder)
fisher_opfname = '%s/s4_cmb_fisher.npy' %(results_folder)

In [ ]:
#execute
cmd1 = 'python3 %s -opfname %s' %(ilc_pgm_name, ilc_opfname)
print('\n######')
print(cmd1)
os.system(cmd1)
cmd2 = 'python3 %s -ilc_fname %s -opfname %s' %(fisher_pgm_name, ilc_opfname, fisher_opfname)
print('\n######')
print(cmd2)
#os.system(cmd2)
print('######')


######
python3 get_ilc_weights_and_residuals.py -opfname results//s4_cmb_ilc.npy


In [ ]:
#now read the Fisher matrix and get constraints
which_spectra = 'delensed'
results_dict = np.load(fisher_opfname, allow_pickle = True).item()
cosmo_params = results_dict['cosmoParams']
fisher_mat_dict = results_dict['fisherGaussian'][0]
fisher_mat = fisher_mat_dict[which_spectra] * fsky

In [ ]:
#fix params and add priors
fisher_mat, cosmo_params = tools.fn_fix_params(fisher_mat, cosmo_params, fix_params)
cosmo_params = np.asarray(cosmo_params)
#add prior
fisher_mat = tools.fn_add_prior(fisher_mat, cosmo_params, prior_dict)

In [ ]:
cov_mat = np.linalg.inv(fisher_mat)
cosmo_constraints = np.diag(cov_mat)**0.5

print('One sigma errors are listed below:\n')
for p, sigma in zip(cosmo_params, cosmo_constraints):
    print('\tParam = %s; %g' %(p, sigma))
sys.exit()